In [9]:
# 동영상을 입력으로하며, 학습된 모델을 이용하여 프레임 단위로 분석하고 예측한 관절 좌표를 생성 및 저장한다.
import os

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # tf.config.experimental.set_visible_devices(gpus[0], "GPU")
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

import deeplabcut
import cv2 
import yaml
import numpy as np
import tqdm.notebook as tqdm
from skimage.util import img_as_ubyte
import time
import json
import shutil

from data_sensor import ini_DLC, get_img_coord, analyze_frames

import angle_out as angle

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

"""
----- param -----
src_file_path : 분석할 동영상 파일 
config_path : 학습된 모델의 yaml 파일 = dlc_cfg
cfg_path : DLC 프로젝트의 yaml 파일 = cfg
----- pose_cfg.yaml에 추가 -----

init_weights: "/drive/samba/private_files/jupyter/DLC/dog1/dlc-models/iteration-0/dog1Mar24-trainset95shuffle1/train/snapshot-30000"
mean_pixel: [123.68, 116.779, 103.939]
weight_decay: 0.0001
pairwise_predict: False
partaffinityfield_predict: False
stride: 8.0
intermediate_supervision: False
dataset_type: imgaug
"""


""" edit. 폴더명 파라미터로 받아서 실행하기
"""

' edit. 폴더명 파라미터로 받아서 실행하기\n'

In [12]:
def analyze_video(src=None, out=None):
    des_out=""
    if isinstance(src, list):        
        for src_file_path in src:
            src_arr = src_file_path.split('/')
            des_out=f"{out}/"
            print(src_arr)
            label = f"{src_arr[-2]}"

            for i in src_arr[2:-1]:
                print(f"소스 경로 출력 > {i}")
                des_out+=f"{i}/"
                if not os.path.isdir(des_out):
                    os.mkdir(des_out)

            print(des_out)

            vedio_name, video_ext = os.path.splitext(os.path.basename(src_file_path))
            print(src_file_path)
            out_path = des_out
            label = des_out+label
    
    data = analyze_frames(src, des_out)
    print(f"file path > {label}")
    print(f"des_out > {des_out}")
    #print("LB : ",type(data),data.shape) # <class 'numpy.ndarray'> (271, 21, 3)
    np.save(label, data)
    
# analyze_video(src=(
#     './data/BODYLOWER/101_bodylower/BODYLOWER_BODYLOWER_frame_102_timestamp_3400.jpg',
#     './data/BODYLOWER/101_bodylower/BODYLOWER_BODYLOWER_frame_108_timestamp_3600.jpg',
#     './data/BODYLOWER/101_bodylower/BODYLOWER_BODYLOWER_frame_114_timestamp_3800.jpg'
# ),
#     out='./label')

#def load_npy_data():
#    np_path='./coco_train02.mov.npy'
#    data = np.load(np_path)
#    print(data)
#    pass

#load_npy_data()


In [13]:
# analyze_video(src ='./data/' + i + j1 , out='./label/' + i+'/' + j1)

for i in os.listdir('./data'):
    if i=='.ipynb_checkpoints':
        continue
        
    if not os.path.exists('./label/' + i):
            os.makedirs('./label/' + i) # label 폴더에 행동 폴더 만들기
            
    for j in os.listdir('./data/'+i):
        # print(f"./label/{i}/{j}")
        f"./data/{i}/{j}"
        # print(j)
        if not os.path.exists('./label/' + i + '/' + j):
            os.makedirs('./label/' + i + '/' + j) # label 폴더안에 행동 폴더 안에 데이터랑 이름이 같은 폴더 만들기
        
        f = open('./label/' + i+'/' + j + '/' + j + '.json', 'w')
        data = []
        data.append({
            "frame" : 1,
            "pose": f"{i}"
        })
        json.dump(data, f, indent=4)
        for x in os.listdir(f'./data/{i}/{j}'):
            print(f"./data/{i}/{j}/{x}")
            analyze_video(src=f"./data/{i}/{j}/{x}", out='/label')
        f.close()   
        




./data/BODYLOWER/183_bodylower/BODYLOWER_BODYLOWER_frame_0_timestamp_0.jpg
전체 길이 : 1.0
초당 프레임 : 25
flag : 12
추가 한 후 0
조건 식 후 : 1
한번 실행하는데 걸리는 시간 : 5.75506 sec
이미지 호출하는데 걸리는 시간 : 0.00476 sec
프리딕션에 걸리는 시간 : 5.75012 sec


UnboundLocalError: local variable 'label' referenced before assignment

In [ ]:
i = 'TEST'
j1 = '1_test'
        
analyze_video(src ='./data/' + i + '/' + j1 , out='./label/' + i+'/' + j1)
        



In [ ]:
# shutil.rmtree('./label', ignore_errors=True)
# 라벨 전체 삭제됨 주의

In [ ]:
# def analyze_video(src=None, out=None):
#     des_out=""
#     if isinstance(src, list):        
#         for src_file_path in src:
#             src_arr = src_file_path.split('/')
#             des_out=f"{out}/"
#             print(src_arr)
#             label = f"{src_arr[-2]}"

#             for i in src_arr[2:-1]:
#                 print(f"소스 경로 출력 > {i}")
#                 des_out+=f"{i}/"
#                 if not os.path.isdir(des_out):
#                     os.mkdir(des_out)

#             print(des_out)

#             vedio_name, video_ext = os.path.splitext(os.path.basename(src_file_path))
#             print(src_file_path)
#             out_path = des_out
#             label = des_out+label
    
#     data = analyze_frames(src, des_out)
#     print(f"file path > {label}")
#     print(f"des_out > {des_out}")
#     #print("LB : ",type(data),data.shape) # <class 'numpy.ndarray'> (271, 21, 3)
#     np.save(label, data)